In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
from torch_geometric.data import Data

2.5.1+cu124
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import zipfile
import os
import numpy as np

# Define the paths to the zip files
male_zip_path = 'generated_male_train_matrices.zip'
female_zip_path = 'generated_female_train_matrices.zip'

# Define extraction paths
male_extract_path = 'male_train_matrices'
female_extract_path = 'female_train_matrices'

# Extract the zip files
with zipfile.ZipFile(male_zip_path, 'r') as zip_ref:
    zip_ref.extractall(male_extract_path)

with zipfile.ZipFile(female_zip_path, 'r') as zip_ref:
    zip_ref.extractall(female_extract_path)

# Now, the NPY files are directly in the extraction folders
male_train_matrices = []
female_train_matrices = []

# Load all NPY files for males directly from the extraction folder
for filename in os.listdir(male_extract_path):
    if filename.endswith('.npy'):
        file_path = os.path.join(male_extract_path, filename)
        male_matrix = np.load(file_path)
        male_train_matrices.append(male_matrix)

# Load all NPY files for females directly from the extraction folder
for filename in os.listdir(female_extract_path):
    if filename.endswith('.npy'):
        file_path = os.path.join(female_extract_path, filename)
        female_matrix = np.load(file_path)
        female_train_matrices.append(female_matrix)

# Check if matrices are loaded correctly
print("Number of male matrices:", len(male_train_matrices))
print("Number of female matrices:", len(female_train_matrices))


Number of male matrices: 1000
Number of female matrices: 1000


In [ ]:
import zipfile
import os
import numpy as np

# Define the paths to the zip files
male_zip_path = 'converted_males_test_matrices.zip'
female_zip_path = 'converted_females_test_matrices.zip'

# Define extraction paths
male_extract_path = 'males_test_matrices'
female_extract_path = 'females_test_matrices'

# Extract the zip files
with zipfile.ZipFile(male_zip_path, 'r') as zip_ref:
    zip_ref.extractall(male_extract_path)

with zipfile.ZipFile(female_zip_path, 'r') as zip_ref:
    zip_ref.extractall(female_extract_path)

# Now, the NPY files are directly in the extraction folders
male_test_matrices = []
female_test_matrices = []

# Load all NPY files for males directly from the extraction folder
for filename in os.listdir(male_extract_path):
    if filename.endswith('.npy'):
        file_path = os.path.join(male_extract_path, filename)
        male_matrix = np.load(file_path)
        male_test_matrices.append(male_matrix)

# Load all NPY files for females directly from the extraction folder
for filename in os.listdir(female_extract_path):
    if filename.endswith('.npy'):
        file_path = os.path.join(female_extract_path, filename)
        female_matrix = np.load(file_path)
        female_test_matrices.append(female_matrix)

# Check if matrices are loaded correctly
print("Number of male matrices:", len(male_test_matrices))
print("Number of female matrices:", len(female_test_matrices))


Number of male matrices: 7
Number of female matrices: 10


In [ ]:
import torch
import torch_geometric
from torch_geometric.data import Data
import networkx as nx
import random


# Convert a matrix to edge list and edge weights
def matrix_to_graph(matrix):
    # Use from_numpy_array instead of from_numpy_matrix
    G = nx.from_numpy_array(matrix, create_using=nx.DiGraph)  # Directed graph
    edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()
    edge_weights = torch.tensor([G[u][v]['weight'] for u, v in G.edges], dtype=torch.float32)
    return edge_index, edge_weights


# Create the graph data for males and females
male_train_graphs = []
female_train_graphs = []

# Process male matrices
for matrix in male_train_matrices:
    edge_index, edge_weights = matrix_to_graph(matrix)
    # Node features are the row vectors of the matrix (connections to all other nodes)
    node_features = torch.tensor(matrix, dtype=torch.float32)  # Directly use the connectivity matrix as features
    # Append data object
    male_train_graphs.append(Data(x=node_features, edge_index=edge_index, edge_attr=edge_weights, y=torch.tensor([0])))  # 0 for male

# Process female matrices
for matrix in female_train_matrices:
    edge_index, edge_weights = matrix_to_graph(matrix)
    node_features = torch.tensor(matrix, dtype=torch.float32)  # Directly use the connectivity matrix as features
    female_train_graphs.append(Data(x=node_features, edge_index=edge_index, edge_attr=edge_weights, y=torch.tensor([1])))  # 1 for female


torch.manual_seed(12345)

# Combine both male and female data
graphs_train = male_train_graphs + female_train_graphs
random.shuffle(graphs_train)

print(len(graphs_train))
print(graphs_train)


2000
[Data(x=[90, 90], edge_index=[2, 6279], edge_attr=[6279], y=[1]), Data(x=[90, 90], edge_index=[2, 6353], edge_attr=[6353], y=[1]), Data(x=[90, 90], edge_index=[2, 6327], edge_attr=[6327], y=[1]), Data(x=[90, 90], edge_index=[2, 6344], edge_attr=[6344], y=[1]), Data(x=[90, 90], edge_index=[2, 6355], edge_attr=[6355], y=[1]), Data(x=[90, 90], edge_index=[2, 6243], edge_attr=[6243], y=[1]), Data(x=[90, 90], edge_index=[2, 6326], edge_attr=[6326], y=[1]), Data(x=[90, 90], edge_index=[2, 6308], edge_attr=[6308], y=[1]), Data(x=[90, 90], edge_index=[2, 6269], edge_attr=[6269], y=[1]), Data(x=[90, 90], edge_index=[2, 6342], edge_attr=[6342], y=[1]), Data(x=[90, 90], edge_index=[2, 6297], edge_attr=[6297], y=[1]), Data(x=[90, 90], edge_index=[2, 6284], edge_attr=[6284], y=[1]), Data(x=[90, 90], edge_index=[2, 6306], edge_attr=[6306], y=[1]), Data(x=[90, 90], edge_index=[2, 6251], edge_attr=[6251], y=[1]), Data(x=[90, 90], edge_index=[2, 6277], edge_attr=[6277], y=[1]), Data(x=[90, 90], ed

In [ ]:
import torch
import torch_geometric
from torch_geometric.data import Data
import networkx as nx
import random


# Convert a matrix to edge list and edge weights
def matrix_to_graph(matrix):
    # Use from_numpy_array instead of from_numpy_matrix
    G = nx.from_numpy_array(matrix, create_using=nx.DiGraph)  # Directed graph
    edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()
    edge_weights = torch.tensor([G[u][v]['weight'] for u, v in G.edges], dtype=torch.float32)
    return edge_index, edge_weights


# Create the graph data for males and females
male_test_graphs = []
female_test_graphs = []

# Process male matrices
for matrix in male_test_matrices:
    edge_index, edge_weights = matrix_to_graph(matrix)
    # Node features are the row vectors of the matrix (connections to all other nodes)
    node_features = torch.tensor(matrix, dtype=torch.float32)  # Directly use the connectivity matrix as features
    # Append data object
    male_test_graphs.append(Data(x=node_features, edge_index=edge_index, edge_attr=edge_weights, y=torch.tensor([0])))  # 0 for male

# Process female matrices
for matrix in female_test_matrices:
    edge_index, edge_weights = matrix_to_graph(matrix)
    node_features = torch.tensor(matrix, dtype=torch.float32)  # Directly use the connectivity matrix as features
    female_test_graphs.append(Data(x=node_features, edge_index=edge_index, edge_attr=edge_weights, y=torch.tensor([1])))  # 1 for female


torch.manual_seed(12345)

# Combine both male and female data
graphs_test = male_test_graphs + female_test_graphs
random.shuffle(graphs_test)

print(len(graphs_test))
print(graphs_test)
torch.save(graphs_test, 'graphs_test.pt')
import shutil

# Create a zip archive of the .pt file
shutil.make_archive('graphs_test', 'zip', '.', 'graphs_test.pt')
from google.colab import files

# Download the zip file
# files.download('graphs_test.zip')

17
[Data(x=[90, 90], edge_index=[2, 5833], edge_attr=[5833], y=[1]), Data(x=[90, 90], edge_index=[2, 6223], edge_attr=[6223], y=[1]), Data(x=[90, 90], edge_index=[2, 6748], edge_attr=[6748], y=[1]), Data(x=[90, 90], edge_index=[2, 6564], edge_attr=[6564], y=[1]), Data(x=[90, 90], edge_index=[2, 5862], edge_attr=[5862], y=[1]), Data(x=[90, 90], edge_index=[2, 6219], edge_attr=[6219], y=[1]), Data(x=[90, 90], edge_index=[2, 6453], edge_attr=[6453], y=[1]), Data(x=[90, 90], edge_index=[2, 6411], edge_attr=[6411], y=[1]), Data(x=[90, 90], edge_index=[2, 6598], edge_attr=[6598], y=[1]), Data(x=[90, 90], edge_index=[2, 6486], edge_attr=[6486], y=[1]), Data(x=[90, 90], edge_index=[2, 6159], edge_attr=[6159], y=[1]), Data(x=[90, 90], edge_index=[2, 6167], edge_attr=[6167], y=[1]), Data(x=[90, 90], edge_index=[2, 6148], edge_attr=[6148], y=[1]), Data(x=[90, 90], edge_index=[2, 6636], edge_attr=[6636], y=[1]), Data(x=[90, 90], edge_index=[2, 5601], edge_attr=[5601], y=[1]), Data(x=[90, 90], edge

In [ ]:
import torch
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F


class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, 64)
        self.conv3 = GCNConv(64, out_channels)

        # Adding Batch Normalization layers
        self.batch_norm1 = nn.BatchNorm1d(64)  # Normalize after first GCN layer
        self.batch_norm2 = nn.BatchNorm1d(64)  # Normalize after second GCN layer
        self.batch_norm3 = nn.BatchNorm1d(out_channels)  # Normalize after third GCN layer

        self.dropout = nn.Dropout(p=0.5)  # Dropout for regularization
        self.fc = torch.nn.Linear(out_channels, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First GCN layer with Batch Normalization and ReLU
        x = self.conv1(x, edge_index)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Second GCN layer with Batch Normalization and ReLU
        x = self.conv2(x, edge_index)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Third GCN layer with Batch Normalization and ReLU
        x = self.conv3(x, edge_index)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Global pooling and output
        x = global_mean_pool(x, data.batch)
        x = self.fc(x)
        x = torch.sigmoid(x)

        return x


# Hyperparameters
in_channels = male_train_matrices[0].shape[0]  # Each matrix is 90x90, so in_channels=90
hidden_channels = 64
out_channels = 64
learning_rate = 0.001
num_epochs = 200

# Create the model
model = GCN(in_channels=in_channels, out_channels=out_channels)
# Define loss function (binary cross-entropy for classification)
loss_fn = nn.BCELoss()
# Define optimizer (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


In [ ]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(graphs_train, batch_size=16, shuffle=True)
test_loader = DataLoader(graphs_test, batch_size=16, shuffle=False)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()  # Clear previous gradients
        # Forward pass through the model
        output = model(batch)
        # Compute the loss
        loss = loss_fn(output.view(-1), batch.y.float())  # Flatten output to match batch.y shape
  # Use .squeeze() to match dimensions
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

Epoch 1/200, Loss: 0.2209155206978321
Epoch 2/200, Loss: 0.06266446509212255
Epoch 3/200, Loss: 0.03549618623778224
Epoch 4/200, Loss: 0.04687826618738473
Epoch 5/200, Loss: 0.023754909599199892
Epoch 6/200, Loss: 0.023260516786016525
Epoch 7/200, Loss: 0.021755100779235363
Epoch 8/200, Loss: 0.015381967584136874
Epoch 9/200, Loss: 0.034491265162825586
Epoch 10/200, Loss: 0.015887549417093395
Epoch 11/200, Loss: 0.021544213734101505
Epoch 12/200, Loss: 0.022616369169903918
Epoch 13/200, Loss: 0.01387321619107388
Epoch 14/200, Loss: 0.015180161811294966
Epoch 15/200, Loss: 0.008992573561612517
Epoch 16/200, Loss: 0.009700714574661106
Epoch 17/200, Loss: 0.020305352720664815
Epoch 18/200, Loss: 0.01627341341227293
Epoch 19/200, Loss: 0.008693715883186086
Epoch 20/200, Loss: 0.015281903458759188
Epoch 21/200, Loss: 0.010513929750304669
Epoch 22/200, Loss: 0.006687855016090907
Epoch 23/200, Loss: 0.004533486949861981
Epoch 24/200, Loss: 0.0062053155296016484
Epoch 25/200, Loss: 0.004743572

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from torch_geometric.loader import DataLoader

# Function to compute accuracy
def compute_accuracy(model, loader):
    model.eval()  # Set to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No gradients needed for evaluation
        for batch in loader:
            output = model(batch)  # Forward pass
            predictions = (output.view(-1) > 0.5).long()  # Convert probabilities to binary labels
            correct += (predictions == batch.y).sum().item()
            total += batch.y.size(0)

    accuracy = correct / total
    return accuracy

# # Function to compute precision, recall, F1 score, and AUC
# def compute_metrics(model, loader):
#     model.eval()  # Set to evaluation mode
#     all_predictions = []
#     all_labels = []
#     all_probabilities = []

#     with torch.no_grad():
#         for batch in loader:
#             output = model(batch)  # Forward pass
#             probabilities = output.view(-1)  # Get probabilities for AUC calculation
#             predictions = (probabilities > 0.5).long()  # Convert probabilities to binary labels
#             all_predictions.append(predictions)
#             all_labels.append(batch.y)
#             all_probabilities.append(probabilities)

#     # Flatten the list of batches into a single tensor
#     all_predictions = torch.cat(all_predictions)
#     all_labels = torch.cat(all_labels)
#     all_probabilities = torch.cat(all_probabilities)

#     # Compute precision, recall, F1 score, and AUC
#     precision = precision_score(all_labels.cpu(), all_predictions.cpu())
#     recall = recall_score(all_labels.cpu(), all_predictions.cpu())
#     f1 = f1_score(all_labels.cpu(), all_predictions.cpu())
#     auc = roc_auc_score(all_labels.cpu(), all_probabilities.cpu())

#     return precision, recall, f1, auc

# # Create data loaders for training and testing


# # Store accuracies for each epoch
# train_accuracies = []
# test_accuracies = []

# # Training loop with accuracy calculation and other metrics
# for epoch in range(num_epochs):
#     model.train()  # Set the model to training mode
#     total_loss = 0

#     for batch in train_loader:
#         optimizer.zero_grad()  # Clear previous gradients
#         output = model(batch)  # Forward pass
#         loss = loss_fn(output.view(-1), batch.y.float())  # Compute loss
#         loss.backward()  # Backpropagation
#         optimizer.step()  # Update weights

#         total_loss += loss.item()

#     # Compute train and test accuracy after each epoch
test_accuracy = compute_accuracy(model, test_loader)
print('GCN:')
print(test_accuracy)
#     test_accuracy = compute_accuracy(model, test_loader)

#     # Compute precision, recall, F1 score, and AUC
#     precision, recall, f1, auc = compute_metrics(model, test_loader)

#     # Store the accuracies
#     train_accuracies.append(train_accuracy)
#     test_accuracies.append(test_accuracy)

#     # Print the evaluation results for each epoch
#     print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}, "
#           f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}, "
#           f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}")

# # Calculate the mean accuracy
# mean_train_accuracy = sum(train_accuracies) / len(train_accuracies)
# mean_test_accuracy = sum(test_accuracies) / len(test_accuracies)

# print(f"Mean Train Accuracy: {mean_train_accuracy:.4f}")
# print(f"Mean Test Accuracy: {mean_test_accuracy:.4f}")


GCN:
0.7058823529411765


In [ ]:
import torch
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from torch_geometric.nn import GATConv, global_mean_pool
import torch.nn.functional as F


class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels, num_heads=4):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, 64, heads=num_heads, concat=True)
        self.conv2 = GATConv(64 * num_heads, out_channels, heads=1, concat=False)

        # Batch Normalization
        self.batch_norm1 = nn.BatchNorm1d(64 * num_heads)
        self.batch_norm2 = nn.BatchNorm1d(out_channels)

        self.dropout = nn.Dropout(p=0.5)  # Dropout for regularization
        self.fc = torch.nn.Linear(out_channels, 1)  # Fully connected output

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First GAT layer with Batch Normalization, ReLU, and Dropout
        x = self.conv1(x, edge_index)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Second GAT layer with Batch Normalization, ReLU, and Dropout
        x = self.conv2(x, edge_index)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Global mean pooling and output
        x = global_mean_pool(x, data.batch)
        x = self.fc(x)
        x = torch.sigmoid(x)

        return x


# Hyperparameters
num_heads = 8  # GAT typically benefits from multiple attention heads
learning_rate = 0.001
num_epochs = 50

# Create the GAT model
gat_model = GAT(in_channels=in_channels, out_channels=out_channels, num_heads=num_heads)
gat_optimizer = torch.optim.Adam(gat_model.parameters(), lr=learning_rate, weight_decay=1e-4)
gat_scheduler = torch.optim.lr_scheduler.StepLR(gat_optimizer, step_size=10, gamma=0.5)
gat_loss_fn = nn.BCELoss()


In [ ]:
# import torch
# import torch.nn as nn
# import torch_geometric.nn as pyg_nn
# from torch_geometric.nn import GATConv

# from torch_geometric.nn import global_mean_pool
# import torch.nn.functional as F

# class GAT(torch.nn.Module):
#     def __init__(self, in_channels, out_channels, heads=8):
#         super(GAT, self).__init__()
#         # First GAT layer
#         self.gat1 = GATConv(in_channels, 64, heads=heads, concat=True)  # Multiple attention heads
#         # Second GAT layer
#         self.gat2 = GATConv(64 * heads, out_channels, heads=1, concat=False)  # Output layer

#         # Batch Normalization layers
#         self.batch_norm1 = nn.BatchNorm1d(64 * heads)  # Normalize after first GAT layer
#         self.batch_norm2 = nn.BatchNorm1d(out_channels)  # Normalize after second GAT layer

#         self.dropout = nn.Dropout(p=0.5)  # Dropout for regularization
#         self.fc = torch.nn.Linear(out_channels, 1)  # Fully connected layer for the output

#     def forward(self, data):
#         x, edge_index = data.x, data.edge_index

#         # First GAT layer with Batch Normalization and ReLU
#         x = self.gat1(x, edge_index)
#         x = self.batch_norm1(x)  # Apply Batch Normalization
#         x = F.relu(x)  # ReLU activation
#         x = self.dropout(x)  # Dropout

#         # Second GAT layer with Batch Normalization and ReLU
#         x = self.gat2(x, edge_index)
#         x = self.batch_norm2(x)  # Apply Batch Normalization
#         x = F.relu(x)  # ReLU activation
#         x = self.dropout(x)  # Dropout

#         # Global pooling and output
#         x = global_mean_pool(x, data.batch)  # Global pooling for the graph
#         x = self.fc(x)  # Final output layer
#         x = torch.sigmoid(x)  # Sigmoid for binary classification

#         return x

# # Hyperparameters
# in_channels = male_train_matrices[0].shape[0]  # Each matrix is 90x90, so in_channels=90
# hidden_channels = 64  # You can adjust this based on the complexity of your dataset
# out_channels = 64  # Typically, this would be a smaller value (e.g., 32 or 64)
# learning_rate = 0.001
# num_epochs = 50
# heads = 8  # Number of attention heads in the first GAT layer

# # Create the model
# model = GAT(in_channels=in_channels, out_channels=out_channels, heads=heads)

# # Define loss function (binary cross-entropy for classification)
# loss_fn = nn.BCELoss()

# # Define optimizer (Adam)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# # Define learning rate scheduler
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)



In [ ]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(graphs_train, batch_size=16, shuffle=True)
test_loader = DataLoader(graphs_test, batch_size=16, shuffle=False)

# Training loop
for epoch in range(num_epochs):
    gat_model.train()  # Set the model to training mode
    total_loss = 0

    for batch in train_loader:
        gat_optimizer.zero_grad()  # Clear previous gradients
        # Forward pass through the model
        output = gat_model(batch)
        # Compute the loss
        loss = gat_loss_fn(output.view(-1), batch.y.float())  # Flatten output to match batch.y shape
  # Use .squeeze() to match dimensions
        # Backward pass and optimize
        loss.backward()
        gat_optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

Epoch 1/50, Loss: 0.13571386083960532
Epoch 2/50, Loss: 0.05136297668516636
Epoch 3/50, Loss: 0.02806380289234221
Epoch 4/50, Loss: 0.024695208881050347
Epoch 5/50, Loss: 0.026283745245076717
Epoch 6/50, Loss: 0.027442645171657204
Epoch 7/50, Loss: 0.013645309565821663
Epoch 8/50, Loss: 0.019081479916116223
Epoch 9/50, Loss: 0.012440885468153282
Epoch 10/50, Loss: 0.03364085392537527
Epoch 11/50, Loss: 0.00803913957020268
Epoch 12/50, Loss: 0.014800522457808257
Epoch 13/50, Loss: 0.015121178262867033
Epoch 14/50, Loss: 0.010005934746004641
Epoch 15/50, Loss: 0.015724976405501367
Epoch 16/50, Loss: 0.02238372562872246
Epoch 17/50, Loss: 0.019858565533999353
Epoch 18/50, Loss: 0.006050260441959836
Epoch 19/50, Loss: 0.0136446287909057
Epoch 20/50, Loss: 0.004334560922929086
Epoch 21/50, Loss: 0.004230949767399579
Epoch 22/50, Loss: 0.010853226637700573
Epoch 23/50, Loss: 0.015397927286074264
Epoch 24/50, Loss: 0.008222078043560031
Epoch 25/50, Loss: 0.008023014597245493
Epoch 26/50, Loss

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from torch_geometric.loader import DataLoader

# Function to compute accuracy
def compute_accuracy(model, loader):
    model.eval()  # Set to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No gradients needed for evaluation
        for batch in loader:
            output = model(batch)  # Forward pass
            predictions = (output.view(-1) > 0.5).long()  # Convert probabilities to binary labels
            correct += (predictions == batch.y).sum().item()
            total += batch.y.size(0)

    accuracy = correct / total
    return accuracy

# # Function to compute precision, recall, F1 score, and AUC
# def compute_metrics(model, loader):
#     model.eval()  # Set to evaluation mode
#     all_predictions = []
#     all_labels = []
#     all_probabilities = []

#     with torch.no_grad():
#         for batch in loader:
#             output = model(batch)  # Forward pass
#             probabilities = output.view(-1)  # Get probabilities for AUC calculation
#             predictions = (probabilities > 0.5).long()  # Convert probabilities to binary labels
#             all_predictions.append(predictions)
#             all_labels.append(batch.y)
#             all_probabilities.append(probabilities)

#     # Flatten the list of batches into a single tensor
#     all_predictions = torch.cat(all_predictions)
#     all_labels = torch.cat(all_labels)
#     all_probabilities = torch.cat(all_probabilities)

#     # Compute precision, recall, F1 score, and AUC
#     precision = precision_score(all_labels.cpu(), all_predictions.cpu())
#     recall = recall_score(all_labels.cpu(), all_predictions.cpu())
#     f1 = f1_score(all_labels.cpu(), all_predictions.cpu())
#     auc = roc_auc_score(all_labels.cpu(), all_probabilities.cpu())

#     return precision, recall, f1, auc

# # Create data loaders for training and testing


# # Store accuracies for each epoch
# train_accuracies = []
# test_accuracies = []

# # Training loop with accuracy calculation and other metrics
# for epoch in range(num_epochs):
#     model.train()  # Set the model to training mode
#     total_loss = 0

#     for batch in train_loader:
#         optimizer.zero_grad()  # Clear previous gradients
#         output = model(batch)  # Forward pass
#         loss = loss_fn(output.view(-1), batch.y.float())  # Compute loss
#         loss.backward()  # Backpropagation
#         optimizer.step()  # Update weights

#         total_loss += loss.item()

#     # Compute train and test accuracy after each epoch
test_accuracy = compute_accuracy(gat_model, test_loader_gat)
print('GAT:')
print(test_accuracy)
#     test_accuracy = compute_accuracy(model, test_loader)

#     # Compute precision, recall, F1 score, and AUC
#     precision, recall, f1, auc = compute_metrics(model, test_loader)

#     # Store the accuracies
#     train_accuracies.append(train_accuracy)
#     test_accuracies.append(test_accuracy)

#     # Print the evaluation results for each epoch
#     print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}, "
#           f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}, "
#           f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}")

# # Calculate the mean accuracy
# mean_train_accuracy = sum(train_accuracies) / len(train_accuracies)
# mean_test_accuracy = sum(test_accuracies) / len(test_accuracies)

# print(f"Mean Train Accuracy: {mean_train_accuracy:.4f}")
# print(f"Mean Test Accuracy: {mean_test_accuracy:.4f}")


ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
import torch
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from torch_geometric.nn import SAGEConv, global_mean_pool
import torch.nn.functional as F


class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, out_channels)

        # Batch Normalization layers
        self.batch_norm1 = nn.BatchNorm1d(hidden_channels)
        self.batch_norm2 = nn.BatchNorm1d(hidden_channels)
        self.batch_norm3 = nn.BatchNorm1d(out_channels)

        self.dropout = nn.Dropout(p=0.5)
        self.fc = torch.nn.Linear(out_channels, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First GraphSAGE layer
        x = self.conv1(x, edge_index)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Second GraphSAGE layer
        x = self.conv2(x, edge_index)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Third GraphSAGE layer
        x = self.conv3(x, edge_index)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Global pooling and output
        x = global_mean_pool(x, data.batch)
        x = self.fc(x)
        x = torch.sigmoid(x)

        return x


# Hyperparameters
in_channels = 90  # Each matrix is 90x90, so in_channels=90
hidden_channels = 64
out_channels = 64
learning_rate = 0.001
num_epochs2 = 100

# Create the model
model3 = GraphSAGE(in_channels=in_channels, hidden_channels=hidden_channels, out_channels=out_channels)
# Define loss function (binary cross-entropy for classification)
loss_fn3 = nn.BCELoss()
# Define optimizer (Adam)
optimizer3 = torch.optim.Adam(model3.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler3 = torch.optim.lr_scheduler.StepLR(optimizer3, step_size=10, gamma=0.5)


In [ ]:
from torch_geometric.loader import DataLoader

train_loader3 = DataLoader(graphs_train, batch_size=16, shuffle=True)
test_loader3 = DataLoader(graphs_test, batch_size=16, shuffle=False)

# Training loop
for epoch in range(num_epochs2):
    model3.train()  # Set the model to training mode
    total_loss = 0

    for batch in train_loader3:
        optimizer3.zero_grad()  # Clear previous gradients
        # Forward pass through the model
        output = model3(batch)
        # Compute the loss
        loss = loss_fn3(output.view(-1), batch.y.float())  # Flatten output to match batch.y shape
  # Use .squeeze() to match dimensions
        # Backward pass and optimize
        loss.backward()
        optimizer3.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs2}, Loss: {total_loss / len(train_loader3)}")

Epoch 1/100, Loss: 0.31124151876568795
Epoch 2/100, Loss: 0.05519972073659301
Epoch 3/100, Loss: 0.0438655938077718
Epoch 4/100, Loss: 0.03419433236308396
Epoch 5/100, Loss: 0.022207931641489268
Epoch 6/100, Loss: 0.015206223753280937
Epoch 7/100, Loss: 0.028032338194083423
Epoch 8/100, Loss: 0.01236983290174976
Epoch 9/100, Loss: 0.01628657115274109
Epoch 10/100, Loss: 0.019057660017861053
Epoch 11/100, Loss: 0.025299216595944016
Epoch 12/100, Loss: 0.02747358604823239
Epoch 13/100, Loss: 0.015665593964979053
Epoch 14/100, Loss: 0.008517505291267298
Epoch 15/100, Loss: 0.009269966468797065
Epoch 16/100, Loss: 0.01684837445442099
Epoch 17/100, Loss: 0.007585410477768164
Epoch 18/100, Loss: 0.01784797082497971
Epoch 19/100, Loss: 0.012831837811507285
Epoch 20/100, Loss: 0.008107375147694256
Epoch 21/100, Loss: 0.01329473547177622
Epoch 22/100, Loss: 0.012196470352704637
Epoch 23/100, Loss: 0.009744180084089749
Epoch 24/100, Loss: 0.007965622042887845
Epoch 25/100, Loss: 0.02184379914763

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from torch_geometric.loader import DataLoader

# Function to compute accuracy
def compute_accuracy(model, loader):
    model.eval()  # Set to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No gradients needed for evaluation
        for batch in loader:
            output = model(batch)  # Forward pass
            predictions = (output.view(-1) > 0.5).long()  # Convert probabilities to binary labels
            correct += (predictions == batch.y).sum().item()
            total += batch.y.size(0)

    accuracy = correct / total
    return accuracy

# # Function to compute precision, recall, F1 score, and AUC
# def compute_metrics(model, loader):
#     model.eval()  # Set to evaluation mode
#     all_predictions = []
#     all_labels = []
#     all_probabilities = []

#     with torch.no_grad():
#         for batch in loader:
#             output = model(batch)  # Forward pass
#             probabilities = output.view(-1)  # Get probabilities for AUC calculation
#             predictions = (probabilities > 0.5).long()  # Convert probabilities to binary labels
#             all_predictions.append(predictions)
#             all_labels.append(batch.y)
#             all_probabilities.append(probabilities)

#     # Flatten the list of batches into a single tensor
#     all_predictions = torch.cat(all_predictions)
#     all_labels = torch.cat(all_labels)
#     all_probabilities = torch.cat(all_probabilities)

#     # Compute precision, recall, F1 score, and AUC
#     precision = precision_score(all_labels.cpu(), all_predictions.cpu())
#     recall = recall_score(all_labels.cpu(), all_predictions.cpu())
#     f1 = f1_score(all_labels.cpu(), all_predictions.cpu())
#     auc = roc_auc_score(all_labels.cpu(), all_probabilities.cpu())

#     return precision, recall, f1, auc

# # Create data loaders for training and testing


# # Store accuracies for each epoch
# train_accuracies = []
# test_accuracies = []

# # Training loop with accuracy calculation and other metrics
# for epoch in range(num_epochs):
#     model.train()  # Set the model to training mode
#     total_loss = 0

#     for batch in train_loader:
#         optimizer.zero_grad()  # Clear previous gradients
#         output = model(batch)  # Forward pass
#         loss = loss_fn(output.view(-1), batch.y.float())  # Compute loss
#         loss.backward()  # Backpropagation
#         optimizer.step()  # Update weights

#         total_loss += loss.item()

#     # Compute train and test accuracy after each epoch
test_accuracy = compute_accuracy(model3, test_loader3)
print('GraphSage:')
print(test_accuracy)
#     test_accuracy = compute_accuracy(model, test_loader)

#     # Compute precision, recall, F1 score, and AUC
#     precision, recall, f1, auc = compute_metrics(model, test_loader)

#     # Store the accuracies
#     train_accuracies.append(train_accuracy)
#     test_accuracies.append(test_accuracy)

#     # Print the evaluation results for each epoch
#     print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}, "
#           f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}, "
#           f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}")

# # Calculate the mean accuracy
# mean_train_accuracy = sum(train_accuracies) / len(train_accuracies)
# mean_test_accuracy = sum(test_accuracies) / len(test_accuracies)

# print(f"Mean Train Accuracy: {mean_train_accuracy:.4f}")
# print(f"Mean Test Accuracy: {mean_test_accuracy:.4f}")


GraphSage:
0.5882352941176471
